# Tree Methods Consulting Project

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('dogfood').getOrCreate()

In [4]:
data = spark.read.csv('/FileStore/tables/dog_food.csv',inferSchema=True,header=True)

In [5]:
data.printSchema()

In [6]:
data.head(10)

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
data_dropna = data.dropna()

In [9]:
assembler = VectorAssembler(inputCols=['A','B','C','D'],outputCol='features')

In [10]:
data_vec = assembler.transform(data_dropna)

In [11]:
train_data,test_data = data_vec.randomSplit([0.7,0.3])

In [12]:
from pyspark.ml.classification import RandomForestClassifier

In [13]:
rfc = RandomForestClassifier(labelCol='Spoiled',featuresCol='features',numTrees=100)

In [14]:
rfc_model = rfc.fit(train_data)

In [15]:
rfc_predictions = rfc_model.transform(test_data)

In [16]:
rfc_model.featureImportances

In [17]:
inputCols = ['A','B','C','D']

In [18]:
spoilingFactorIndex = rfc_model.featureImportances.toArray().argmax()

In [19]:
spoilingFactor = inputCols[spoilingFactorIndex]

In [20]:
spoilingFactor